In [2]:
import json
import pandas as pd 
import re

#### Explore
##### how is the each rel looks like in the data

In [ ]:
gold_json_file = "/users/the/NER_MTB/0_breast_train_gold_dct.json"
gold_ids_file = "/users/the/NER_MTB/chemoTimelines2024_train_dev_labeled/subtask1/All_Patient_IDs/breast_train_patient_ids.txt"

save_file = "./breast_train_chemo_time_rel_accuracy.csv"

gold_ids = []
with open(gold_ids_file, "r") as infile:
    lines = infile.readlines()
gold_ids.extend([id.strip() for id in lines])
sorted(gold_ids)
with open(gold_json_file, "r") as infile:
    gold_dct = json.load(infile)

def _extract_info(lst):
    if lst[0][0] < lst[1][0]: 
        new_lst = [lst[0][0], lst[1][1]]
    else: 
        new_lst = [lst[1][0], lst[0][1]]
    return new_lst

def _truncate_text(lst, raw_note):
    start_ind, end_ind = int(lst[0]), int(lst[1])
    truncated_text = raw_note[start_ind:end_ind]
    return truncated_text

def _helper_gold(gold_dct, pat_id, filename):
    print("GOLD:")
    # not always: source: chemo, target: time
    gold_chemo_id_lst, gold_time_id_lst = gold_dct[pat_id][filename]["chemo_time_rel"]["source_id"], gold_dct[pat_id][filename]["chemo_time_rel"]["target_id"]
    gold_tlink_lst = gold_dct[pat_id][filename]["chemo_time_rel"]["rel_type"]
    gold_dct_str = gold_dct[pat_id][filename]["doc_create_time"]["ment"]
    if (gold_dct_str is not None) and (gold_dct_str[-1] == "\n"):
        gold_dct_str = gold_dct_str[:-1]
    gold_tuple= []
    rel_wi_spans=[]
    for gold_chemo_id, gold_time_id in zip(gold_chemo_id_lst, gold_time_id_lst):
        if gold_chemo_id in gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"]:
            gold_chemo_ind, gold_timex_ind = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"].index(gold_chemo_id), gold_dct[pat_id][filename]["pair_wise"]["time"]["ment_id"].index(gold_time_id)
        else:
            gold_chemo_ind, gold_timex_ind = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment_id"].index(gold_time_id), gold_dct[pat_id][filename]["pair_wise"]["time"]["ment_id"].index(gold_chemo_id)
        gold_chemo, gold_timex =  gold_dct[pat_id][filename]["pair_wise"]["chemo"]["ment"][gold_chemo_ind], gold_dct[pat_id][filename]["pair_wise"]["time"]["ment"][gold_timex_ind]
        gold_tuple.append([gold_chemo, gold_timex])
        chemo_span, timex_span = gold_dct[pat_id][filename]["pair_wise"]["chemo"]["span"][gold_chemo_ind], gold_dct[pat_id][filename]["pair_wise"]["time"]["span"][gold_timex_ind]
        rel_wi_spans.append([chemo_span, timex_span])
    # sorted_gold_tuple = sorted(gold_tuple, key=lambda x: x[1])
    gold_df = pd.DataFrame(gold_tuple, columns=["chemo", "rel_raw_time"])
    gold_df.insert(0, "DCT", gold_dct_str)
    gold_df["original_DCT"] = gold_df["DCT"]
    gold_df["DCT"] = pd.to_datetime(gold_df["DCT"], format="%Y%m%d")
    gold_df["DCT"] = gold_df["DCT"].fillna(gold_df["original_DCT"])
    gold_df["tlink"] = gold_tlink_lst
    gold_df.drop(columns=["original_DCT"], inplace=True)
    gold_df["rel_wi_spans"] = rel_wi_spans
    gold_df["rel_wi_span"] = gold_df["rel_wi_spans"].apply(_extract_info)
    raw_note = gold_dct[pat_id][filename]["raw_note"]
    gold_df["rel_wi_text"] = gold_df["rel_wi_span"].apply(lambda x: _truncate_text(x, raw_note))
    gold_df.drop(columns=["rel_wi_spans","rel_wi_span"], inplace=True, axis=1)
    if gold_df.shape[0] == 0:
        print("The GOLD file didn't find any chemo related time")
    else:
        print(gold_df)
        return(gold_df)



df = pd.DataFrame() 
for pat_id in gold_ids:
    print(f"~~~~~~~~{pat_id}~~~~~~~~")
    if pat_id in gold_dct.keys():
            for filename in sorted(list(gold_dct[pat_id].keys())):
                print("~~~~~~~~filename: ", pat_id+"_"+filename, "~~~~~~~~")
                gold_df = _helper_gold(gold_dct, pat_id, filename)
                if gold_df is None or gold_df.shape[0] == 0:
                    continue
                gold_df.insert(0, "idf", pat_id+"_"+filename)
                df = pd.concat([df, gold_df])
    else: 
        continue
df.to_csv(save_file)